In [70]:
import pandas as pd
from IPython.display import Image
import matplotlib.pyplot as plt
import altair as alt
from vega_datasets import data

%matplotlib inline
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

pd.set_option('display.float_format', lambda x: '%.3f' % x)

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Definitions

**Research ODA definition:** "Official Development Assistance (ODA) is the term used by Development Assistance Committee (DAC) members to refer to what most people would call aid. To be counted as ODA, public money must be given outright or loaned on concessional (non-commercial) terms, and be used to support the welfare or development of developing countries." (Source FAQ page, more details on [definition & coverage](https://www.oecd.org/dac/financing-sustainable-development/development-finance-standards/officialdevelopmentassistancedefinitionandcoverage.htm))

![ExampleODAtypes](resources/ADV2020xls-Screenshot-ODAtypes.png)

**OECD ODA recipients** The DAC List of ODA Recipients shows all countries and territories eligible to receive official development assistance (ODA). These consist of all low and middle income countries based on gross national income (GNI) per capita as published by the World Bank, with the exception of G8 members, EU members, and countries with a firm date for entry into the EU. The list also includes all of the Least Developed Countries (LDCs) as defined by the United Nations (UN). The DAC revises the list every three years. Countries that have exceeded the high-income threshold for three consecutive years at the time of the review are removed. (...) The next review of the DAC List will take place in 2023. ([Source](https://www.oecd.org/dac/financing-sustainable-development/development-finance-standards/daclist.htm))

**DAC** Development Assistance Committee 

**Bilateral aid** represents flows from official (government) sources directly to official sources in the recipient country.

**Multilateral aid** represents core contributions from official (government) sources to multilateral agencies where it is then used to fund the multilateral agencies’ own programmes.

In some cases, a donor can contract with a multilateral agency to deliver a programme or project on its behalf in a recipient country. Such cases are typically counted as bilateral flows and are often referred to as Bi/Multi.

**Official donors** DAC + Non-DAC + multilateral; excludes private donors which according to OECD *should* be excluded

## Conversion 
### between constant USD 2017 (Chinese data) und current prices million USD (OECD data)

* **Current prices to constant ones** *The following table shows the annual rates of the Consumer Price Index. To convert current dollars of any year to constant dollars, divide them by the index of that year and multiply them by the index of the base year you choose (remember that the numerator contains the index value of the year you want to move to). For example, using this index, `$10,000 in 1997 would be 12,622 in 2008 constant dollars ($10,000 × 114.1/  90.4 = $12,622)`.* Source: [Canadian Statistics Office](https://www150.statcan.gc.ca/n1/pub/75f0011x/2011001/notes/analytical-analytiques-eng.htm)
    * is the Consumer Price Index different for each country or is there a global one?


* ***Current dollars** is a term describing income in the year in which a person, household, or family receives it. For example, the income someone received in 1989 unadjusted for inflation is in current dollars.**Constant or real dollars** are terms describing income after adjustment for inflation. The Dictionary of Business and Economics defines constant dollar values and real income as shown below.* Source: [US Census Bureau](https://www.census.gov/topics/income-poverty/income/guidance/current-vs-constant-dollars.html)

# Checks

## In 2020, are there African countries *not* eligible for ODA?

In [71]:
eligible_countries = pd.read_csv('../data/OECD-ODA-eligible-country-2020.csv')

eligible_countries.head()

,ODA_eligible_country,iso3code,numericISOcode,eligibility
0,Afghanistan,AFG,4.000,yes
1,Democratic Peoples Republic of Korea,PRK,408.000,yes
2,Armenia,ARM,51.000,yes
3,Albania,ALB,8.000,yes
4,Angola,AGO,24.000,yes


In [72]:
countries = alt.topo_feature(data.world_110m.url, "countries")
countries

UrlData({
  format: TopoDataFormat({
    feature: 'countries',
    type: 'topojson'
  }),
  url: 'https://cdn.jsdelivr.net/npm/vega-datasets@v1.29.0/data/world-110m.json'
})

In [73]:
import altair as alt
from vega_datasets import data

countries = alt.topo_feature(data.world_110m.url, "countries")

projection_type="naturalEarth1"
scale_value=240
center_values=[25,10]

base = alt.Chart(countries).mark_geoshape(
    fill='#D8DDE2',
    stroke='white'
).properties(
    width=1200,
    height=800,
).project(
    type=projection_type,
    center=center_values,
    scale=scale_value
)

chart = alt.Chart(countries).mark_geoshape().encode(
    alt.Color('eligibility:N', type ='ordinal', scale=alt.Scale(domain=['yes'],range=["#00A5FF"])),
    tooltip=["ODA_eligible_country:N", 'eligibility:N']
).transform_lookup(
    lookup="id",
    from_=alt.LookupData(eligible_countries,'numericISOcode',['ODA_eligible_country','eligibility'])
).project(
    type=projection_type,
    center=center_values,
    scale=scale_value
).properties(
    width=1200,
    height=800,
    title='Countries eligible for ODA in 2020'
)

eligble_map = base + chart
eligble_map

alt.LayerChart(...)

# DATA INSPECTION

In [74]:
df = pd.DataFrame()

for year in range(2002,2020,1):
    filename = '../data/ODA-yearly/ODA-'+str(year)+'-DAC2a.csv'
    df_temp = pd.read_csv(filename)
    df_temp['year'] = year
    df = pd.concat([df, df_temp])

df = df.drop('FakeColumn', axis=1)
df.head()

,Recipient,"Official Donors, Total","DAC Countries, Total","Non-DAC Countries, Total","Multilateral Agencies, Total","Private Donors, Total",Australia,Austria,Belgium,Canada,Czech Republic,Denmark,Finland,France,Germany,Greece,Hungary,Iceland,Ireland,Italy,Japan,Korea,Luxembourg,Netherlands,New Zealand,Norway,Poland,Portugal,Slovak Republic,Slovenia,Spain,Sweden,Switzerland,United Kingdom,United States,EU Institutions,Azerbaijan,Bulgaria,Croatia,Cyprus,Estonia,Israel,Kazakhstan,Kuwait,Latvia,Liechtenstein,Lithuania,Malta,Qatar,Romania,Russia,Saudi Arabia,Chinese Taipei,Thailand,Timor-Leste,Turkey,United Arab Emirates,Other donor countries,"International Monetary Fund, Total",IMF (Concessional Trust Funds),"Regional Development Banks, Total","African Development Bank, Total",African Development Bank [AfDB],African Development Fund [AfDF],"Asian Development Bank, Total",Asian Development Bank [AsDB],"Inter-American Development Bank, Total",Inter-American Development Bank [IDB],Asian Infrastructure Investment Bank [AIIB],Caribbean Development Bank [CarDB],Council of Europe Development Bank [CEB],European Bank for Reconstruction and Development [EBRD],Islamic Development Bank [IsDB],"United Nations, Total",Food and Agriculture Organisation [FAO],International Atomic Energy Agency [IAEA],IFAD,International Labour Organisation [ILO],UNAIDS,UNDP,UNECE,UNEP,UNFPA,UNHCR,UNICEF,UN Institute for Disarmament Research [UNIDIR],UN Peacebuilding Fund [UNPBF],UNRWA,UNTA,WFP,World Health Organisation [WHO],World Tourism Organisation [UNWTO],"World Bank Group, Total","World Bank, Total",International Bank for Reconstruction and Development [IBRD],International Development Association [IDA],International Finance Corporation [IFC],"Other Multilateral, Total",Adaptation Fund,Arab Bank for Economic Development in Africa [BADEA],Arab Fund (AFESD),Center of Excellence in Finance [CEF],Central Emergency Response Fund [CERF],Climate Investment Funds [CIF],Global Alliance for Vaccines and Immunization [GAVI],Global Environment Facility [GEF],Global Fund,Global Green Growth Institute [GGGI],Green Climate Fund [GCF],Montreal Protocol,Nordic Development Fund [NDF],OPEC Fund for International Development [OPEC Fund],OSCE,Other Multilaterals,Arcadia Fund,Arcus Foundation,BBVA Microfinance Foundation,Bernard van Leer Foundation,Bill & Melinda Gates Foundation,Carnegie Corporation of New York,Charity Projects Ltd (Comic Relief),Children's Investment Fund Foundation,Citi Foundation,Conrad N. Hilton Foundation,David & Lucile Packard Foundation,Ford Foundation,Gatsby Charitable Foundation,Gordon and Betty Moore Foundation,Grameen Crédit Agricole Foundation,H&M Foundation,IKEA Foundation,Jacobs Foundation,John D. & Catherine T. MacArthur Foundation,La Caixa Banking Foundation,Laudes Foundation,LEGO Foundation,Margaret A. Cargill Foundation,MasterCard Foundation,MAVA Foundation,McKnight Foundation,MetLife Foundation,Michael & Susan Dell Foundation,Oak Foundation,"Omidyar Network Fund, Inc.",Rockefeller Foundation,Wellcome Trust,William & Flora Hewlett Foundation,World Diabetes Foundation,Dutch Postcode Lottery,Norwegian Postcode Lottery,People's Postcode Lottery,Swedish Postcode Lottery,"G7 Countries, Total","DAC-EU Countries, Total","United Postcode Lotteries, Total",year,Donor(s),Unnamed: 0
0,All Recipients,NaN,58654.230,3264.430,17112.800,NaN,988.740,520.130,1071.510,2004.280,45.430,1643.240,462.200,5486.170,5324.410,276.170,NaN,12.650,397.780,2332.130,9282.950,278.750,146.760,3338.020,121.820,1696.090,14.260,322.580,6.680,NaN,1712.300,2011.060,938.890,4929.160,13290.070,5448.120,NaN,NaN,NaN,NaN,1.250,130.890,NaN,20.380,1.500,NaN,1.640,NaN,NaN,NaN,NaN,2477.750,NaN,NaN,NaN,72.960,558.060,NaN,901.620,901.620,876.150,714.560,99.040,615.520,NaN,NaN,NaN,NaN,NaN,62.800,NaN,43.900,54.890,3142.650,NaN,NaN,148.470,NaN,NaN,274.680,NaN,NaN,310.170,633.490,566.870,NaN,NaN,392.200,466.160,350.610,NaN,NaN,6163.420,6163.420,NaN,6163.420,NaN,580.840,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [75]:
df_melted = df.melt(id_vars=['Recipient','year'])
df_melted.head()

,Recipient,year,variable,value
0,All Recipients,2002,"Official Donors, Total",NaN
1,"All Developing Countries, Total",2002,"Official Donors, Total",60966.210
2,"All Multilateral Recipients, Total",2002,"Official Donors, Total",18065.250
3,"Africa, Total",2002,"Official Donors, Total",21368.300
4,"North of Sahara, Total",2002,"Official Donors, Total",2106.580


In [76]:
df_melted[(df_melted['Recipient']=='Afghanistan')&(df_melted['variable']=='Germany')]

,Recipient,year,variable,value
64390,Afghanistan,2002,Germany,92.570
64665,Afghanistan,2003,Germany,82.100
64940,Afghanistan,2004,Germany,9.610
65215,Afghanistan,2005,Germany,17.280
65490,Afghanistan,2006,Germany,10.910
65765,Afghanistan,2007,Germany,17.250
66040,Afghanistan,2008,Germany,10.130
66315,Afghanistan,2009,Germany,17.970
66865,Afghanistan,2011,Germany,539.300
67140,Afghanistan,2012,Germany,515.540


In [77]:
supp1 = pd.read_csv('../data/ODA-supplementary-donortype.csv')
supp1.head()

,donor,donor_type
0,"Official Donors, Total",summary
1,"DAC Countries, Total",summary
2,"Non-DAC Countries, Total",summary
3,"Multilateral Agencies, Total",summary
4,"Private Donors, Total",summary


In [78]:
df_melted = df_melted.merge(supp1, left_on='variable', right_on='donor')
df_melted.head()

,Recipient,year,variable,value,donor,donor_type
0,All Recipients,2002,"Official Donors, Total",NaN,"Official Donors, Total",summary
1,"All Developing Countries, Total",2002,"Official Donors, Total",60966.210,"Official Donors, Total",summary
2,"All Multilateral Recipients, Total",2002,"Official Donors, Total",18065.250,"Official Donors, Total",summary
3,"Africa, Total",2002,"Official Donors, Total",21368.300,"Official Donors, Total",summary
4,"North of Sahara, Total",2002,"Official Donors, Total",2106.580,"Official Donors, Total",summary


In [79]:
df_melted = df_melted[['Recipient', 'year','value', 'donor', 'donor_type']]
df_melted.columns= (['Recipient', 'year','currentPrices_Million_USD', 'donor', 'donor_type'])

In [80]:
df_melted.head()

,Recipient,year,currentPrices_Million_USD,donor,donor_type
0,All Recipients,2002,NaN,"Official Donors, Total",summary
1,"All Developing Countries, Total",2002,60966.210,"Official Donors, Total",summary
2,"All Multilateral Recipients, Total",2002,18065.250,"Official Donors, Total",summary
3,"Africa, Total",2002,21368.300,"Official Donors, Total",summary
4,"North of Sahara, Total",2002,2106.580,"Official Donors, Total",summary


In [81]:
overview_developingCountries = df_melted[(df_melted['donor_type']=='summary')&(df_melted['donor']=='Official Donors, Total')&(df_melted['Recipient']=='All Developing Countries, Total')]
overview_developingCountries

,Recipient,year,currentPrices_Million_USD,donor,donor_type
1,"All Developing Countries, Total",2002,60966.210,"Official Donors, Total",summary
276,"All Developing Countries, Total",2003,70525.190,"Official Donors, Total",summary
551,"All Developing Countries, Total",2004,54785.710,"Official Donors, Total",summary
826,"All Developing Countries, Total",2005,83112.970,"Official Donors, Total",summary
1101,"All Developing Countries, Total",2006,77622.450,"Official Donors, Total",summary
1376,"All Developing Countries, Total",2007,73734.610,"Official Donors, Total",summary
1651,"All Developing Countries, Total",2008,87128.240,"Official Donors, Total",summary
1926,"All Developing Countries, Total",2009,83968.370,"Official Donors, Total",summary
2476,"All Developing Countries, Total",2011,139449.770,"Official Donors, Total",summary
2751,"All Developing Countries, Total",2012,133751.600,"Official Donors, Total",summary


In [82]:
overview_africa = df_melted[(df_melted['donor_type']=='summary')&(df_melted['donor']=='Official Donors, Total')&(df_melted['Recipient']=='Africa, Total')]

In [83]:
overview_ssa_africa = df_melted[(df_melted['donor_type']=='summary')&(df_melted['donor']=='Official Donors, Total')&(df_melted['Recipient']=='South of Sahara, Total')]

In [84]:
all_devC=alt.Chart(overview_developingCountries).mark_bar(color='#6D7780').encode(
    x='year:N',
    y='currentPrices_Million_USD:Q'
)

african_c=alt.Chart(overview_africa).mark_bar(color='#002D5A').encode(
    x='year:N',
    y='currentPrices_Million_USD:Q'
)

ssa_c = alt.Chart(overview_ssa_africa).mark_bar(color='#00A5FF').encode(
    x='year:N',
    y='currentPrices_Million_USD:Q'
)

layered = alt.layer(all_devC, african_c, ssa_c)

layered.properties(width = 300)

alt.LayerChart(...)

In [85]:
df_melted['donor_type'].value_counts()

bilateral       257400
private         198000
multilateral    133650
subgroup        133650
summary          34650
Name: donor_type, dtype: int64

In [86]:
df_melted.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 757350 entries, 0 to 757349
Data columns (total 5 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   Recipient                  715275 non-null  object
 1   year                       757350 non-null  int64 
 2   currentPrices_Million_USD  203751 non-null  object
 3   donor                      757350 non-null  object
 4   donor_type                 757350 non-null  object
dtypes: int64(1), object(4)
memory usage: 34.7+ MB


In [87]:
donor_types = df_melted.groupby(by='donor_type')['currentPrices_Million_USD'].sum().reset_index()

In [88]:
donor_types=donor_types.drop(index = 3)

In [89]:
donor_types

,donor_type,currentPrices_Million_USD
0,bilateral,15703295.850
1,multilateral,3189056.010
2,private,16194914.710
4,summary,31871263.640


In [90]:
sum_money_OECD_donors = donor_types['currentPrices_Million_USD'].sum()

In [91]:
donor_types['share_donorType'] = [donor_money / sum_money_OECD_donors for donor_money in donor_types['currentPrices_Million_USD']]

In [92]:
donor_types

,donor_type,currentPrices_Million_USD,share_donorType
0,bilateral,15703295.850,0.235
1,multilateral,3189056.010,0.048
2,private,16194914.710,0.242
4,summary,31871263.640,0.476


In [93]:
alt.Chart(donor_types).mark_bar(height = 100).encode(
    alt.X('sum(currentPrices_Million_USD):Q',stack='normalize'),
    color='donor_type:N'
).properties(
    height=100
)

alt.Chart(...)

## Chinese data

In [94]:
cf = pd.read_csv('../data/AidData_excerpt_aggregates_all_years.csv', delimiter=';')
cf.head()

,\AidData TUFF Project ID,Recommended For Aggregates,Umbrella,Financier Country,Recipient,Recipient Region,Commitment Year,Title,Status,Intent,Flow Type,Concessional,Flow Class,Sector Code,Sector Name,Funding Agencies,Funding Agencies Type,Amount (Original Currency),Original Currency,Amount (Constant USD2017),Amount (Nominal)
0,53631,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China provides humanitarian aid to Afghanistan...,Completion,Development,Grant,Yes,ODA-like,720,EMERGENCY RESPONSE,Unspecified Chinese Government Institution,Government Agency,137100000.000,AFN,2015410.812,2015410.812
1,53632,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates equipment to Kabul University,Completion,Development,Grant,Yes,ODA-like,110,EDUCATION,Unspecified Chinese Government Institution,Government Agency,NaN,NaN,NaN,NaN
2,53633,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates $1 million USD to UNHCR for huma...,Completion,Development,Grant,Yes,ODA-like,720,EMERGENCY RESPONSE,Unspecified Chinese Government Institution,Government Agency,1000000.000,USD,1000000.000,1000000.000
3,53634,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,Chinese Government provides RMB 180 million gr...,Completion,Development,Grant,Yes,ODA-like,110,EDUCATION,China Ministry of Commerce,Government Agency,18000000.000,CNY,2663728.108,2663728.108
4,53636,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates desks and chairs to China-Afghan...,Completion,Representational,Grant,Yes,OOF-like,110,EDUCATION,Chinese Embassy,Government Agency,NaN,NaN,NaN,NaN


In [95]:
cf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10849 entries, 0 to 10848
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   \AidData TUFF Project ID    10849 non-null  int64  
 1   Recommended For Aggregates  10849 non-null  object 
 2   Umbrella                    10849 non-null  object 
 3   Financier Country           10849 non-null  object 
 4   Recipient                   10849 non-null  object 
 5   Recipient Region            10849 non-null  object 
 6   Commitment Year             10849 non-null  int64  
 7   Title                       10849 non-null  object 
 8   Status                      10849 non-null  object 
 9   Intent                      10849 non-null  object 
 10  Flow Type                   10849 non-null  object 
 11  Concessional                10849 non-null  object 
 12  Flow Class                  10849 non-null  object 
 13  Sector Code                 108

### Chinese financying by flow type

In [96]:
cf['Flow Class'].value_counts()

ODA-like                    7567
OOF-like                    2781
Vague (Official Finance)     501
Name: Flow Class, dtype: int64

In [97]:
cf.groupby(by='Flow Class')['Amount (Constant USD2017)'].sum()

Flow Class
ODA-like                   100558302427.056
OOF-like                   681961752028.526
Vague (Official Finance)    60373239964.850
Name: Amount (Constant USD2017), dtype: float64

In [98]:
cf_flowTypes = cf.groupby(by='Flow Class')['Amount (Constant USD2017)'].sum().reset_index()

In [99]:
cf_flowTypes_sum = cf_flowTypes['Amount (Constant USD2017)'].sum()

In [100]:
cf_flowTypes['share_pct_flowType'] = [flow_type_amount / cf_flowTypes_sum for flow_type_amount in cf_flowTypes['Amount (Constant USD2017)'] ]

In [101]:
cf_flowTypes

,Flow Class,Amount (Constant USD2017),share_pct_flowType
0,ODA-like,100558302427.056,0.119
1,OOF-like,681961752028.526,0.809
2,Vague (Official Finance),60373239964.850,0.072


In [102]:
alt.Chart(cf_flowTypes).mark_bar(width=250).encode(
    alt.Y('sum(Amount (Constant USD2017)):Q', stack='normalize'),
    color='Flow Class:N'

).properties(width=600)

alt.Chart(...)

### Apply OECD ODA-eligibility to Chinese data

In [103]:
eligible_countries.head()

,ODA_eligible_country,iso3code,numericISOcode,eligibility
0,Afghanistan,AFG,4.000,yes
1,Democratic Peoples Republic of Korea,PRK,408.000,yes
2,Armenia,ARM,51.000,yes
3,Albania,ALB,8.000,yes
4,Angola,AGO,24.000,yes


In [104]:
countrycodesAidData = pd.read_csv('../data/country-and-continent-codes-list-forAidData.csv', delimiter=';')

In [105]:
cfm = cf.merge(countrycodesAidData[['country','ISO3code']], left_on='Recipient', right_on='country')

In [106]:
cfm.head()

,\AidData TUFF Project ID,Recommended For Aggregates,Umbrella,Financier Country,Recipient,Recipient Region,Commitment Year,Title,Status,Intent,Flow Type,Concessional,Flow Class,Sector Code,Sector Name,Funding Agencies,Funding Agencies Type,Amount (Original Currency),Original Currency,Amount (Constant USD2017),Amount (Nominal),country,ISO3code
0,53631,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China provides humanitarian aid to Afghanistan...,Completion,Development,Grant,Yes,ODA-like,720,EMERGENCY RESPONSE,Unspecified Chinese Government Institution,Government Agency,137100000.000,AFN,2015410.812,2015410.812,Afghanistan,AFG
1,53632,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates equipment to Kabul University,Completion,Development,Grant,Yes,ODA-like,110,EDUCATION,Unspecified Chinese Government Institution,Government Agency,NaN,NaN,NaN,NaN,Afghanistan,AFG
2,53633,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates $1 million USD to UNHCR for huma...,Completion,Development,Grant,Yes,ODA-like,720,EMERGENCY RESPONSE,Unspecified Chinese Government Institution,Government Agency,1000000.000,USD,1000000.000,1000000.000,Afghanistan,AFG
3,53634,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,Chinese Government provides RMB 180 million gr...,Completion,Development,Grant,Yes,ODA-like,110,EDUCATION,China Ministry of Commerce,Government Agency,18000000.000,CNY,2663728.108,2663728.108,Afghanistan,AFG
4,53636,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates desks and chairs to China-Afghan...,Completion,Representational,Grant,Yes,OOF-like,110,EDUCATION,Chinese Embassy,Government Agency,NaN,NaN,NaN,NaN,Afghanistan,AFG


In [107]:
cfmm = cfm.merge(eligible_countries, left_on='ISO3code',right_on='iso3code')

In [108]:
len(cfmm)

9842

In [109]:
cfmm.head()

,\AidData TUFF Project ID,Recommended For Aggregates,Umbrella,Financier Country,Recipient,Recipient Region,Commitment Year,Title,Status,Intent,Flow Type,Concessional,Flow Class,Sector Code,Sector Name,Funding Agencies,Funding Agencies Type,Amount (Original Currency),Original Currency,Amount (Constant USD2017),Amount (Nominal),country,ISO3code,ODA_eligible_country,iso3code,numericISOcode,eligibility
0,53631,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China provides humanitarian aid to Afghanistan...,Completion,Development,Grant,Yes,ODA-like,720,EMERGENCY RESPONSE,Unspecified Chinese Government Institution,Government Agency,137100000.000,AFN,2015410.812,2015410.812,Afghanistan,AFG,Afghanistan,AFG,4.000,yes
1,53632,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates equipment to Kabul University,Completion,Development,Grant,Yes,ODA-like,110,EDUCATION,Unspecified Chinese Government Institution,Government Agency,NaN,NaN,NaN,NaN,Afghanistan,AFG,Afghanistan,AFG,4.000,yes
2,53633,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates $1 million USD to UNHCR for huma...,Completion,Development,Grant,Yes,ODA-like,720,EMERGENCY RESPONSE,Unspecified Chinese Government Institution,Government Agency,1000000.000,USD,1000000.000,1000000.000,Afghanistan,AFG,Afghanistan,AFG,4.000,yes
3,53634,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,Chinese Government provides RMB 180 million gr...,Completion,Development,Grant,Yes,ODA-like,110,EDUCATION,China Ministry of Commerce,Government Agency,18000000.000,CNY,2663728.108,2663728.108,Afghanistan,AFG,Afghanistan,AFG,4.000,yes
4,53636,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates desks and chairs to China-Afghan...,Completion,Representational,Grant,Yes,OOF-like,110,EDUCATION,Chinese Embassy,Government Agency,NaN,NaN,NaN,NaN,Afghanistan,AFG,Afghanistan,AFG,4.000,yes


In [110]:
cf_odaEligible = cfmm[cfmm['eligibility']=='yes']

In [111]:
len(cf_odaEligible)

9842

### Unify unit: convert current prices to constant ones

In [112]:
# currentPrices_Million_USD_year * (CPI2017 / CPP_year)
# https://www.census.gov/topics/income-poverty/income/guidance/current-vs-constant-dollars.html
cpi_2017 = 361.0
cpi_2020 = 381.2
cpis = [{'year':2000, 'cpi':252.9},{'year':2001, 'cpi':260.1},{'year':2002, 'cpi':264.2},{'year':2003, 'cpi':270.2},{'year':2004, 'cpi':277.5},{'year':2005, 'cpi':286.9},{'year':2006, 'cpi':296.2},{'year':2007, 'cpi':304.6},{'year':2008, 'cpi':316.3},{'year':2009, 'cpi':315.2},{'year':2010, 'cpi':320.4},{'year':2011, 'cpi':330.5},{'year':2012, 'cpi':337.5},{'year':2013, 'cpi':342.5},{'year':2014, 'cpi':348.3},{'year': 2013, 'cpi': 342.5},{'year': 2014, 'cpi': 348.3},{'year': 2015, 'cpi': 348.9}, {'year': 2016, 'cpi': 353.4},{'year': 2017, 'cpi': 361.0}, {'year': 2018, 'cpi': 369.8}, {'year': 2019, 'cpi': 376.5}, {'year':2020, 'cpi': 381.2}]

cpidf = pd.DataFrame(cpis)
cpidf

,year,cpi
0,2000,252.900
1,2001,260.100
2,2002,264.200
3,2003,270.200
4,2004,277.500
5,2005,286.900
6,2006,296.200
7,2007,304.600
8,2008,316.300
9,2009,315.200


### Filter to include bilateraland multilateral donors

In [113]:
dfn = pd.read_csv('../data/OECD-SSA-OOF_ODA.csv')
dfn.head()

,year,Current Prices (USD millions),Flow Class,donor
0,2002,17763.500,ODA,Official Donors
1,2003,24288.163,ODA,Official Donors
2,2004,27253.009,ODA,Official Donors
3,2005,34326.733,ODA,Official Donors
4,2006,81264.320,ODA,Official Donors


In [114]:
dfn_constant = pd.DataFrame()

for year in range(2002,2020,1):
    df_melted_temp = dfn[dfn['year']==year]
    cpi_year = cpidf[cpidf['year']==year]['cpi'].iloc[0]
    df_melted_temp['constant2020_million_USD'] = df_melted_temp['Current Prices (USD millions)'] * (cpi_2020/cpi_year)
    dfn_constant= pd.concat([dfn_constant,df_melted_temp])

/var/folders/fk/_qbrt9ys18b6s2yb_kqblv1xt9_xg3/T/ipykernel_13910/2631004586.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_melted_temp['constant2020_million_USD'] = df_melted_temp['Current Prices (USD millions)'] * (cpi_2020/cpi_year)


In [115]:
dfn_constant

,year,Current Prices (USD millions),Flow Class,donor,constant2020_million_USD
0,2002,17763.500,ODA,Official Donors,25630.001
18,2002,215.279,OOF,Official Donors,310.614
1,2003,24288.163,ODA,Official Donors,34265.906
19,2003,374.109,OOF,Official Donors,527.796
2,2004,27253.009,ODA,Official Donors,37437.286
20,2004,408.078,OOF,Official Donors,560.574
3,2005,34326.733,ODA,Official Donors,45609.448
21,2005,982.710,OOF,Official Donors,1305.713
4,2006,81264.320,ODA,Official Donors,104584.601
22,2006,579.062,OOF,Official Donors,745.234


### Same with Chineses data

In [116]:
cf_odaEligible.head()

,\AidData TUFF Project ID,Recommended For Aggregates,Umbrella,Financier Country,Recipient,Recipient Region,Commitment Year,Title,Status,Intent,Flow Type,Concessional,Flow Class,Sector Code,Sector Name,Funding Agencies,Funding Agencies Type,Amount (Original Currency),Original Currency,Amount (Constant USD2017),Amount (Nominal),country,ISO3code,ODA_eligible_country,iso3code,numericISOcode,eligibility
0,53631,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China provides humanitarian aid to Afghanistan...,Completion,Development,Grant,Yes,ODA-like,720,EMERGENCY RESPONSE,Unspecified Chinese Government Institution,Government Agency,137100000.000,AFN,2015410.812,2015410.812,Afghanistan,AFG,Afghanistan,AFG,4.000,yes
1,53632,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates equipment to Kabul University,Completion,Development,Grant,Yes,ODA-like,110,EDUCATION,Unspecified Chinese Government Institution,Government Agency,NaN,NaN,NaN,NaN,Afghanistan,AFG,Afghanistan,AFG,4.000,yes
2,53633,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates $1 million USD to UNHCR for huma...,Completion,Development,Grant,Yes,ODA-like,720,EMERGENCY RESPONSE,Unspecified Chinese Government Institution,Government Agency,1000000.000,USD,1000000.000,1000000.000,Afghanistan,AFG,Afghanistan,AFG,4.000,yes
3,53634,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,Chinese Government provides RMB 180 million gr...,Completion,Development,Grant,Yes,ODA-like,110,EDUCATION,China Ministry of Commerce,Government Agency,18000000.000,CNY,2663728.108,2663728.108,Afghanistan,AFG,Afghanistan,AFG,4.000,yes
4,53636,Yes,No,China (People's Republic of),Afghanistan,Asia,2017,China donates desks and chairs to China-Afghan...,Completion,Representational,Grant,Yes,OOF-like,110,EDUCATION,Chinese Embassy,Government Agency,NaN,NaN,NaN,NaN,Afghanistan,AFG,Afghanistan,AFG,4.000,yes


## Filter for ODA-like / OOF-like data

In [117]:
selection_cf = cf_odaEligible[(cf_odaEligible['Flow Class']=='ODA-like')|(cf_odaEligible['Flow Class']=='OOF-like')]

In [118]:
cf_grouped = selection_cf.groupby(by=['Recipient', 'Commitment Year', 'Flow Class', 'ISO3code'])['Amount (Constant USD2017)'].sum().reset_index()
cf_grouped

,Recipient,Commitment Year,Flow Class,ISO3code,Amount (Constant USD2017)
0,Afghanistan,2000,ODA-like,AFG,1113800.692
1,Afghanistan,2001,ODA-like,AFG,2446247.143
2,Afghanistan,2002,ODA-like,AFG,15609533.866
3,Afghanistan,2003,ODA-like,AFG,14994850.208
4,Afghanistan,2004,ODA-like,AFG,63438647.712
...,...,...,...,...,...
2367,Zimbabwe,2014,OOF-like,ZWE,186003981.060
2368,Zimbabwe,2015,ODA-like,ZWE,22904189.937
2369,Zimbabwe,2015,OOF-like,ZWE,1374669654.314
2370,Zimbabwe,2016,ODA-like,ZWE,29804740.570


### rework cf data from Constant USD2017 to Constant USD2020

In [119]:
# currentPrice_year = constant price / (cpi_2017 /cpy_year) 
# constant price 2020 = currentPrice_year * (cpi_2020 /cpy_year)

In [120]:
cf_grouped_constant = pd.DataFrame()

for year in range(2000,2018,1):
    cf_grouped_temp = cf_grouped[cf_grouped['Commitment Year']==year]
    cpi_year = cpidf[cpidf['year']==year]['cpi'].iloc[0]
    cf_grouped_temp['currentPrices_Million_USD'] = cf_grouped_temp['Amount (Constant USD2017)'] / (cpi_2017 /cpi_year) /1000000
    cf_grouped_temp['constant2020_million_USD'] = cf_grouped_temp['currentPrices_Million_USD'] * (cpi_2020/cpi_year)
    cf_grouped_constant= pd.concat([cf_grouped_constant,cf_grouped_temp])

/var/folders/fk/_qbrt9ys18b6s2yb_kqblv1xt9_xg3/T/ipykernel_13910/927310507.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf_grouped_temp['currentPrices_Million_USD'] = cf_grouped_temp['Amount (Constant USD2017)'] / (cpi_2017 /cpi_year) /1000000
/var/folders/fk/_qbrt9ys18b6s2yb_kqblv1xt9_xg3/T/ipykernel_13910/927310507.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf_grouped_temp['constant2020_million_USD'] = cf_grouped_temp['currentPrices_Million_USD'] * (cpi_2020/cpi_year)


In [121]:
cf_grouped_constant.head()

,Recipient,Commitment Year,Flow Class,ISO3code,Amount (Constant USD2017),currentPrices_Million_USD,constant2020_million_USD
0,Afghanistan,2000,ODA-like,AFG,1113800.692,0.780,1.176
22,Albania,2000,ODA-like,ALB,2673121.661,1.873,2.823
41,Algeria,2000,ODA-like,DZA,0.000,0.000,0.000
59,Angola,2000,ODA-like,AGO,0.000,0.000,0.000
146,Bangladesh,2000,ODA-like,BGD,145215208.830,101.731,153.341


### Development of donations to each recipient over time

In [122]:
cf_grouped_constant.head()

,Recipient,Commitment Year,Flow Class,ISO3code,Amount (Constant USD2017),currentPrices_Million_USD,constant2020_million_USD
0,Afghanistan,2000,ODA-like,AFG,1113800.692,0.780,1.176
22,Albania,2000,ODA-like,ALB,2673121.661,1.873,2.823
41,Algeria,2000,ODA-like,DZA,0.000,0.000,0.000
59,Angola,2000,ODA-like,AGO,0.000,0.000,0.000
146,Bangladesh,2000,ODA-like,BGD,145215208.830,101.731,153.341


In [123]:
regions = pd.read_csv('../data/country-and-continent-codes-list.csv', delimiter=';')
regions.head()

,continent,region,subregion,country,numericISO,ISO3code,ISO2code,otherGroups
0,Asia,Southern Asia,Southern Asia,Afghanistan,4.000,AFG,AF,LDC LLDC
1,Europe,Southern Europe,Southern Europe,Albania,8.000,ALB,AL,NaN
2,Americas,Northern America,Northern America,Antarctica,10.000,ATA,AQ,NaN
3,Africa,Northern Africa,Northern Africa,Algeria,12.000,DZA,DZ,NaN
4,Oceania,Polynesia,Polynesia,American Samoa,16.000,ASM,AS,SIDS


In [124]:
cf_grouped_constant_m= cf_grouped_constant.merge(regions[['ISO3code', 'region']], left_on='ISO3code', right_on='ISO3code')



In [125]:
cf_grouped_constant_m.head()

,Recipient,Commitment Year,Flow Class,ISO3code,Amount (Constant USD2017),currentPrices_Million_USD,constant2020_million_USD,region
0,Afghanistan,2000,ODA-like,AFG,1113800.692,0.780,1.176,Southern Asia
1,Afghanistan,2001,ODA-like,AFG,2446247.143,1.763,2.583,Southern Asia
2,Afghanistan,2002,ODA-like,AFG,15609533.866,11.424,16.483,Southern Asia
3,Afghanistan,2003,ODA-like,AFG,14994850.208,11.223,15.834,Southern Asia
4,Afghanistan,2004,ODA-like,AFG,63438647.712,48.765,66.988,Southern Asia


In [126]:
cf_grouped_constant_m_ssa = cf_grouped_constant_m[cf_grouped_constant_m['region']=='Sub-Saharan Africa']

In [127]:
cftimeline = cf_grouped_constant_m_ssa.groupby(by=['Commitment Year','Flow Class'])['constant2020_million_USD'].sum().reset_index()
cftimeline.head()

,Commitment Year,Flow Class,constant2020_million_USD
0,2000,ODA-like,259.309
1,2000,OOF-like,158.405
2,2001,ODA-like,2105.367
3,2001,OOF-like,93.570
4,2002,ODA-like,704.148


In [128]:
cftimeline['year_parsed']=pd.to_datetime(cftimeline['Commitment Year'], format='%Y')

In [129]:
alt.Chart(cftimeline).mark_bar().encode(
    alt.Color('Flow Class', scale = alt.Scale(domain=['ODA-like','OOF-like'], range=['#00A5FF','#002D5A'])),
    x='year_parsed:T',
    y='sum(constant2020_million_USD)',

)

alt.Chart(...)

In [130]:
dfn_constant['year_parsed']=pd.to_datetime(dfn_constant['year'], format='%Y')

In [131]:
alt.Chart(dfn_constant).mark_bar().encode(
    alt.Color('Flow Class', scale = alt.Scale(domain=['ODA','OOF'], range=['#00A5FF','#002D5A'])),
    x='year_parsed:T',
    y='sum(constant2020_million_USD)',

)

alt.Chart(...)